In [ ]:
import socket
import time
import random
import threading
import Utils as ut
import Statistics as st

sn = 0  # Sequence number
TIMEOUT_LIMIT = 2  # Timeout for resending frames
copy_sn, copy_data, can_send = 0, "", True

sender = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sender.bind(st.ADDR)

def send_frame(conn, data):
    global sn, can_send, copy_sn, copy_data, time1
    
    # Send frame if allowed
    if can_send:
        frame = ut.noisychannel(ut.makeFrame(sn, data))
        print(f"[SENDING] Frame: {frame}")
        conn.send(frame.encode())

        # Store frame for resending
        copy_sn, copy_data, can_send = sn, data, False
        time1 = time.time()
        sn += 1

        recv_ack(conn)

    # Resend on timeout
    while not can_send:
        resend_frame(conn)

def resend_frame(conn):
    global time1
    frame = ut.noisychannel(ut.makeFrame(copy_sn, copy_data))
    print(f"[RESENDING] Frame: {frame}")
    time1 = time.time()
    conn.send(frame.encode())
    recv_ack(conn)

def recv_ack(conn):
    global can_send
    try:
        conn.settimeout(TIMEOUT_LIMIT)
        ack_frame = conn.recv(20).decode()
        ack_no, _, _, data = ut.receiveFrame(ack_frame)

        if ack_no == copy_sn and data == '11110000':
            print(f"[ACK RECEIVED] ACK {ack_no}")
            can_send = True
    except socket.timeout:
        print("---[TIMEOUT]---")

def start():
    sender.listen()
    print(f"[LISTENING] Server is listening on {st.HOST_IP}")
    conn, addr = sender.accept()
    print(f"[CONNECTED] Connected to {addr}")

    while True:
        data = input('[INPUT] Enter data to send: ')
        send_frame(conn, data)

        if data == 'q':
            print("[CLOSING] Closing the sender...")
            conn.close()
            break

start()
